# Tableau Dashboard Link

You can follow along using my Tableau Dashboard for this analysis here:<a href="https://public.tableau.com/profile/max53497276#!/"> Tableau Profile for Max Mellman</a>

# Conversion Rate Analysis

# Key Question 1:
I'd like to see the website's performance over the course of the first twelve months. What's the percentage change in session to order conversation rate (CVR) on a monthly basis?

<img src="Images/13A-window_function.PNG" width="50%" />

I’ll first grab the 'created_at' column to organize my data at the monthly level. Then call the 'count' function on the 'website_session_id' column to aggregate those sessions. Since I want to know the CVR between sessions to orders I left joined the 'orders' table and aggregated the 'order_id' column and calculated the monthly CVR accordingly. I'll need to calculate the percentage change from month to month in a seperate query, so I'll make this a temporary table. (Note, in PostgreSQL integer values are default when using division and integer division truncates the result towards zero. Thus, casting at least the numerator or denominator as a 'numeric' value provides the accurate results. I'll be using this method throughout my analysis.) 

<img src="Images/13B-window_funct.PNG" width='50%' />

Next, I'll grab the 'conversion_rate' column from the temporary table and use the 'lag' function to pull the previous month's conversation rates creating a new column called 'perct_change' which contains those prior monthly rates. Then, I'll transform the data table containing the window function into a subquery and grab both 'perct_change' and 'conv_rate' columns to calculate monthly percentage change. Lastly, both columns are adjacent to each other allowing me to calculate the percentage change in CVR using simple division and subtraction. 

# Conclusion:
In the month of June and September, increases in CVR of 22% and 17% respectively were realized and are performance outliers in relation to the other months. I will take an analytical approach to identify the reasons for these outsized results and what caused them to occur. These finding will frame the rest of my analysis.

# TRAFFIC SOURCE ANALYSIS

I'd like to get a better understanding of the website design so I'll examine the existing pages limiting my query to the first year.
<img src="Images/AAA2.PNG" width='50%' />
The site introduced two new pages, '/lander-1' and '/billing-2'. Usually a new landing page indicates an attempt to reduce a high bounce rate on the site and improve quality traffic. Additionally, a new billing page indicates an attempt to increase orders and improve the overall experience for the user. The launch dates for these pages coincide with the outsized results in CVR from the prior CVR analysis. For this reason, I'll base my future analysis on the reasons for these additions and their effect on the sites performance. 

# Key Question 2: 
This website has paid traffic and organic traffic as evidenced by the ‘utm_campaign’ and ‘utm_source’ columns. Generally changes in web traffic to a site can be marked by the way a company decides to spend on paid search. I’d like to know where the bulk of the website traffic is coming from?
<img src="Images/1-Bulk_Traffic_Source.PNG" width='75%' />
I’ll perform a ‘count’ aggregation on the 'website_session_id' column and group by source, campaign, and referrer.

# Conclusion:
Users searching with 'gsearch' through the 'nonbrand' campaign produced the most traffic by a significant margin compared to the other paid and organic traffic (‘null’ data). I’ll use these conditions in my ‘where’ clause for the majority of my future queries since this represents the most pertinent source of traffic to the site.

# TRAFFIC SOURCE TRENDING
Since 'gseach' 'nonbrand' traffic represents the most pertinent source of data for my analysis, I’d like to know what the 'gsearch' 'nonbrand' trended session volume is by week? I’ll create a date range of approximately two months starting from the earliest date.
<img src="Images/3-bid_changes_traffic.PNG" width='50%' />
Since I can't group an aggregated measure, I need to 'extract' the week from the desired date range first and use that metric in my 'group by' clause. Then, using the 'min' function call on the same 'created_at' column in order to just show the week corresponding to the extracted 'year_week' column. This is a workaround for manipulating dates in PostgreSQL since I can only pull the week number and not the week date. In mySQL, it is a straightforward process in that I could just pull the 'week' date from the timestamp and group by that same column. I'll be using this method to call dates often for the rest of my analysis.
<h1>Conclusion:</h1>

Based on this query, it looks like 'gsearch' 'nonbrand' sessions saw a reduction in traffic volume by nearly 50%. 

# TRAFFIC CONVERSION RATES
# Key Question 3:
How well is ‘gsearch’ ’nonbrand’ traffic driving sales? 

I’ll examine the weekly CVR based on a date range from where the drop off in session volume occurred, around mid-April.
<img src="Images/2-Session_to_order_cvr.PNG" width='100%' />
From the ‘website_sessions’ table, I’ll left join the ‘orders’ table and aggregate both columns to calculate a CVR to understand how much revenue the paid campaigns are driving.
# Conclusion: 
A conversion rate of 2.9% for paid traffic was perhaps too low for a new business looking for a higher ROI. Unlike a mature retail business with economies of scale, a newer business needs to see greater return on their investments since their forecast models are more susceptible to change. Obviously, ROI requirements vary drastically based on the industry and sector being examined. Ultimately, companies need to see a positive ROI to ensure their campaign is a worthwhile endeavor. 

Regarding cost per click, a low conversion rate can lead to a reduction in bids for a campaign and lead to lower traffic thus the near 50% reduction in traffic volume. With sessions decreasing however, an increase in CVR should result.

# TRAFFIC SOURCE SEGMENT TRENDING
# Key Question 4:
Since examining the changes in weekly trended sessions volume for ‘gsearch’ ‘nonbrand’ traffic led to insights on CVR, I’d like to pull the weekly trends for both desktop and mobile to see if there are any patterns in session volume that would indicate bid changes due to undesirable CVR. 

I'll start with a date range from mid-April to early June. I'll use a 'case' statement to count the sessions by device type, and have the data aggregated at the week-date level. 

<img src="Images/5-traffic_source_segment_trending.PNG" width='75%' />
# Conclusion:
It looks like mobile has been pretty flat to down in session volume, but desktop started increasing in volume in late-May. This could be due to bid changes made based on conversion analysis. 

# TRAFFIC SOURCE BID OPTIMATIZATION
# KEY QUESTION 5:
Session volume for desktop started to trend noticeably higher in late-May, so I’d like to know what the CVR was for both device types before that shift in session volume occurred? Perhaps this analysis will reveal the reason for the change in session volumes. 

I'd like to see the CVR at the device type level, so I'll left join the 'orders' table to the 'website_sessions' table. I call the 'count' functions on both sessions and orders to calculate the CVR and aggregate those metrics by device type.
<img src="Images/4-conv_rate_device_type.PNG" width='100%' />
# Conclusion:
Desktop performance is significantly better than on mobile. This is a crucial insight that most likely caused the jump in paid traffic for desktop. As a result, the company optimizes spend to increase sales for the specific traffic segment.


# IDENTIFYING TOP WEBSITE AND LANDING PAGES
# Key Question 6:
I’d like to know more about the performance of the website pages before the ‘lander-1’ landing page was introduced in mid-June. First, what were the most viewed website pages? And second, what are the top landing pages if there are more than just the home page? 
<img src="Images/6-Identifying_top_website_pages.PNG" width='35%' />
# Conclusion:
It appears the homepage, the products page, and the Mr. Fuzzy page get the bulk of traffic.

Next, I'll pull all entry pages and rank them on entry volume. I want to confirm where most of the sessions are hitting the site. I assume it would be the home page but I must query to be sure. To find top entry pages, I’ll limit to just the first page a user sees during a given session by using the ‘min’ function on the ‘website_pageview_id’ column. For clarity, I’ll create a temporary table that isolates just the first ‘website_pageview_id’ for each corresponding ‘website_session_id’.
<img src="Images/7A-temp table.PNG" width='35%' />
Lastly, I left joined the ‘website_pageviews’ table to the temporary table to link the first pageview_id’s in the ‘first_pv’ column to their corresponding ‘pageview url’ column. An aggregate function on the ‘sessions_hitting_page’ column from the temporary table along with a group by for the ‘landing_page_url’ column yields the following result.
<img src="Images/7B-sessions_hitting_page.PNG" width='75%' />
# Conclusion:
My assumption was correct in that all traffic comes in through the homepage.
Now, I should analyze its bounce rate to confirm the reason for the new landing page as well as an A/B split test to confirm which page improves the site’s performance.



# LANDING PAGE PERFORMANCE & TESTING
# Key Question 7:
Since all of the sesssion volume traffic is directed to the '/home' page, and based on my initial findings of a new landing page being introduced on '2012-06-09', I'd like to know what the bounce rate was for the '/home' page before the new landing page was introduced?

To analyze landing page performance and compare multiple pages, I again use temporary tables.
Similiar to my previous query, I'll find the first pageview for the relevant sessions. 

--Step 1: Find the first ‘website_pageview_id’ for relevant sessions.
<img src="Images/8A-firstpageviews.PNG" width='50%' />

--Step 2: Identifying the landing page of each session.<br>
Next, I’ll bring in the landing page, like last time, but restrict to home only. It's redundant in this case, since all traffic is to the homepage anyway. This is still necessary as most websites will have more than just a single landing page. When I bring in the 'pageview_url' from 'website_pageviews' table I perform a left join on 'website_pageview_id' to 'min_pageview_id' as this is the most pertinent data to match as it corresponds directly to the 'pageview_url' column in the 'website_pageviews' table.
<img src="Images/8B-sessions_with_home_landing_page.PNG" width='75%' />

--Step 3: Counting pageviews for each session, to identify "bounces".<br>
I'm taking the website_session_id's and landing_page from the 'sessions_w_home_landing_page' table and performing a left join with the original 'website_pageviews' table on the 'website_session_id' column. This will allow me to identify which 'website_session_id' had multiple pageviews and which sessions only had one pageview. 
<img src="Images/8C2-behind the scenes_before aggregation.PNG" width='70%' />
Then, I'll perform a group by on 'website_session_id' from the 'website_sessions' table' and using the ‘count’ function on the ‘website_pageview_id’ column which will highlight the difference in session activity. The following image shows which sessions had multiple pageviews and the sessions that only had a single pageview.
<img src="Images/8C3-without_bounced_sessions.PNG" width='70%' />

--Step 4: Summarizing by counting total sessions and bounced sessions.<br>
Lastly, to get a table of 'bounced_sessions' I'll limit the table to just show bounced sessions by assigning the aggregated metric used for 'website_session_id' equal to 1 in the 'having' clause. The following image shows a count of pageviews per session that limits it to just 'bounced_sessions'.
<img src="Images/8C-bounced_sessions.PNG" width='70%' />
By preserving the website sessions id's in the 'sessions_w_home_landing_page' table and left joining the 'bounced_sessions' table it becomes clear that the 'null' values present in the image below represent non-bounced sessions whereas sessions that appear visible are the bounced sessions that are being matched through the left join. 
<img src="Images/8D-showing_bounced_vs_non_bounced_sessions.PNG" width='70%' />
RED: Bounced Session<br>
GREEN: Non-bounced Session

At this stage, I'll perform the aggregation fuction 'count' on both ‘website_session_id’ columns from the 'bounced_sessions' table and 'sessions_w_home_landing_page' table and and include the conversion rate for the 'home' page
<img src="Images/8E-Bounce_rate_landing_page.PNG" width='75%' />
# Conclusion:
At almost 60% the bounce rate for the home page appears high, and it's near the upper end of the average range compared to industry standards. Nonetheless, this should be high quality traffic considering this is paid search through the 'nonbrand' ‘gsearch’ campaign. 

# ANALYZING LANDING PAGE TESTS
# Key Question 8:
I'd like to know if the bounce rate improved with the addition of the '/lander-1' landing page after its introduction?

I'll be conducting an A/B test against '/home' versus '/lander-1' to see if the new page created better results. I'll use the date from the 'created_at' column or a pageview id from the 'website_pageview_id' column to mark the first appearance of '/lander-1' on the website. This will ensure a fair date range for comparison which is crucial to the analysis. I'll frame my test from that date or pageview id to '2012-07-28'. This date represents two months of data which should be sufficient to render a fair comparison.

--Step 1: Find out when the new page ‘/lander-1’ launched.<br>
First, I'll be using the 'first_pageview_id' from this query to establish from where the analysis should start. This will be a more precise measurement rather than using the date as several hundred entries exist in any given day. All pageviews after this point to the date specified above are fair game.
<img src="Images/9A-first_appearance_lander-1.PNG" width='50%' />
--Step 2: Finding the first ‘website_pageview_id’ for relevant sessions.<br>
<img src="Images/9B-first_pageviews_lander_home.PNG" width='75%' />
This query returned 4576 rows which includes only sessions that came through paid search, i.e. 'nonbrand' and 'gsearch', as written in the 'where' clause. This was performed with an inner join between the tables as I needed the matching data for both ‘website_session_id’ columns in each table.<br>
--Step 3: Identifying the landing page of each session.<br>
Similiar to the last analysis, I've made the previous table a temporary table so that I can left join the 'website_pageviews' table on the 'website_pageview_id' column to bring in the 'pageview_url' data into my analysis. This will be shown on the image below.
<img src="Images/9C-landing_page_w_home_lander.PNG" width='75%' />
Looking through the 'landing_page' there's a good mix of '/lander-1' and '/home' which means the introduction of the '/lander-1' page is working to randomize the data and it verifies the experiment is working correctly.<br>
--Step 4: Identify the bounced sessions.<br>
Next, I want to count the pageviews for each session to identify "bounces". In order to determine which sessions had multiple pageviews I need to do a left join to the original 'website_pageviews' table to grab the 'website_pageview_id' column. This column contains unfiltered data so putting this into my query will expand the 'website_session_id' column from the created table in order to match up to every corresponding 'website_pageview_id' brought in from the joined table. The image below shows the duplicate sessions.
<img src="Images/9D-create_multiple_session.PNG" width='80%' />
Applying the ‘count’ function on the 'website_pageview_id' column will aggregate the results and reveal bounced sessions as '1' and non-bounced sessions appearing as '2' or greater since new pageview id's are incrementally assigned for every new view on the website. 
<img src="Images/9E-1_showing_nonbounced_sessions.PNG" width='75%' />
RED: Bounced sessions<br>
GREEN: Non-bounced sessions

I want to isolate the bounced sessions so I will take my aggregate metric on the 'website_pageview_id' column and set that equal to 1 in the having caluse. The query is shown below:
<img src="Images/9E-bounced_sessions.PNG" width='75%' />
--Step 5: Summarizing total sessions and bounced session by landing page.<br>
The query below shows the two newly created tables left joined together to preserve all website sessions that landed on home and lander, and joined with all of the bounced sessions which either appear as 'null', meaning there was no matched value from our bounced table, and 'not null' meaning it was a bounced session as it found a match in the bounced sessions table. The image below shows this:
<img src="Images/9F-1_null_is_nonbounced.PNG" width='75%' />
Finally, applying the 'count' function to both 'bounced_sessions' and 'sessions' as well as calculating the resulting conversation rate yields the desired result. Both pages in this case had similiar volume lending validity to the comparison as both rates carry similiar weight.
<img src="Images/9F-2_final_conversion.PNG" width='75%' />
# Conclusion:
It appears that the new landing page, '/lander-1', has a better bounce rate of 53% compared to the previous bounce rate of 58%. Redirecting all new and current campaigns and nonbrand paid traffic pointing to the new lander will optimize the overall performance of the website for the business.

# QUALIFIED TRAFFIC
# KEY QUESTION 9:
Lander-1 has a lower bounce rate, but does it lead to more qualified traffic?

I’ll create a funnel analysis that looks at sessions landing on 'home' versus '/lander-1' to compare their traffic volumes. I use the same date range as my previous analysis for the '/lander-1' bounce rate.
<img src="Images/15A.PNG" width='100%' />
Starting from the 'website_sessions' table, I'll left join the 'website_pageviews' table so I can find out which version of the pages consumers saw and how far they made it in the conversion funnel. I'm limiting only to paid traffic, using 'gsearch' and 'nonbrand'. I'll use 'case' statements to create flags informing me on how far each session made it through the conversion funnel. In the queried data, I have each session and it's corresponding pageview, and the flag corresponding to that pageview.

Transforming this table into a subquery, I'll pull the 'website_session_id' column and apply the 'max' function on each 'case' statement to get the aggregated view of each session's pageview activity.
<img src="Images/15B.PNG" width='100%' />
I made the data table above into a temporay table. I'll use the 'saw_homepage' and 'saw_custom_lander' column to create the data for the comparison of the two different funnels. 

Once again, using a 'case' statement' from the temparary table, I'll create a flag when a session corresponds to 'saw_homepage' or 'saw_custom_lander' and use that 'case' statement in the 'group by' clause to bring my data to the desired level. Then creating another series of 'case' statements, I'll use the 'count' function for each session that made it to a particular page using the flags as my condition which corresponds to each pageview seen. 
<img src="Images/15C.PNG" width='75%' />
Lastly, I transform the 'made_it' flags into click-through rates using simple division as shown below.
<img src="Images/15D.PNG" width='75%' />
# Conclusion:
The funnel anaylsis reveals that the new '/lander-1' page is increasing click through rates to  the billing page by 5%. This means the new custom landing page is bringing more qualified traffic to the site leading to more orders and an increase in top line revenue growth.

# LANDING PAGE TREND ANALYSIS
# Key Question 10:
I'd like to verifiy if the new landing page was used instead of the old homepage moving through the 2012 year?

To analyze this, I'll create a weekly trend analysis looking at total sessions for the old landing page and the new landing page. I'll pick a wide date range to ensure the results reveal a concrete change.

I'll begin my query by retrieving the 'website_session_id' column from the 'website_sessions' table and left joining the 'website_pageviews' table to retrieve the 'pageview_id' column. I'll perform a 'min' function on the 'pageview_id' column as well as a 'count' function. I'll also make be using an adquate date range to represent the landing page change as well as only use paramaters from the highest traffic volume based on prior analysis.
<img src="Images/10A_weekly_trend_home_lander.PNG" width='75%' />
In this result, I have all of the website_session_id's for the range specified, the first pageview that was seen, and a count of pageviews corresponding to the session activity. Now, I want to bring in the pageview_url's corresponding to the 'first_pageview' column and also the date these sessions were created. I need the date in order to create the weekly trend analysis.
<img src="Images/10B_sesions_w_counts_lander_and_created_at.PNG" width='75%' />
Finally, I'll bring everything together using 'case' statements.
<img src="Images/10C_final_pic.PNG" width='75%' />
# Conclusion:
There's a very clear trend of volume traffic being redirected to the new landing page as evidenced by the decresed volume of home_sessions. On '2012-07-30' volume traffic was entirely redircted to the new landing page and the bounce rate was reduced by nearly 10% from the beginning of the analysis. This clearly answers my initial question of why '/lander-1' was introduced.

# BUILDING CONVERSION FUNNELS
There was also a new billing page introduced, called 'billing-2', on '2012-09-10. There must have been a steep drop off in the coversion funnel for the website at the prior billing page. 

# KEY QUESTION 11:
I'd like to understand further where the website lost their paid traffic between the new '/lander-1' page and placing an order?

I will build a full conversion funnel covering a date range of 1 month prior to the new billing page. I'll analyze how many customers made it to each step starting with /lander-1 and build the funnel all the way to the thank you page from '2012-08-05' to '2012-09-05'

First, I'd like to peform a query to see how many pages exist after the landing page. I'm using the date range specified above which starts well after the introduction of the new landing page and examines about one month worth of data
<img src="Images/11A-typical_conversion_flow.PNG" width='35%' />
Per my analysis, there are six pages commonly viewed after the landing page. I'll build my funnel analysis to include these six pages.

I'll use 'case' statements to flag each 'pageview_url' with a '1' if it exists for the given 'website_session_id' or '0' if no value exists for the given 'website_session_id'. I'll group the data at the 'website_session_id' and 'pageview_url' level since I'd like to see how each user is interacting with the site.
<img src="Images/11B-Flow_of_sessions_thru_pages.PNG" width='75%' />
I transformed the query above into a subquery, and called the 'max' function for each column and grouped by the 'website_session_id' column to make the analysis interpreted at the 'website_session_id' level. Now I can see how each user is interacting with the site.
<img src="Images/11B2-condensed_flow_of_sessions.PNG" width='75%' />
The table for the subquery is below. It shows each sessions pageview activity on the site. Now that the 'max' function has aggregated each metric per website session, I can see the bounced sessions only contain a single '1' value for a single row, whereas multiple '1' values indicate a retained session.   
<img src="Images/11B2-output.PNG" width='75%' />
RED: Bounced session<br>
Green: Non-bounced session<br>

Then, I create a temporary table from the query above and create 'case' statements for each pageview_url column.
<img src="Images/11C-sum_pageviews.PNG" width='75%' />
Next, I aggregated the 'website_session_id' column if the 'case' statement was able to find the flagged pageview_url as indicated by a '1' in the row. I called the 'count' function for each website_session_id corresponding to a flagged pageview_url.<br>

Finally, I performed a conversion analysis on the data.
<img src="Images/11D-rates_output.PNG" width='75%' />
# Conclusion: 
Based on the conversion rates for the funnel analysis, when a user gets to the billing page, only 43% of the users click through to the 'thankyou' page. It's makes sense that a new billing page was introduced to help facilitate a higher conversion rate upon landing on this vital step of the funnel.

# A/B TEST: BILLING-2 VS BILLING-1
# Key Question 12:
Similar to the split A/B test I did for the '/home' and '/lander-1' landing pages, I'd like to perform a test comparing the previous 'billing-1' page against the new 'billing-2' page to see if there was a difference in conversion rates.

In order to frame a fair fight analysis between these two billing pages, I located the first time the '/billing-2' page was seen on the site using its 'pageview_id' to frame the analysis. The date range for this analysis will be at this 'pageview_id' to the last day of the 2012 year.
<img src="Images/12A-reference_billing2_page_id.PNG" width='75%' />
If the billing page is clicked through, then a resulting order_id would be generated in the 'orders' table. I'll left join the 'orders' table to my query to find the sessions that had orders.
<img src="Images/12B-sessions_w_orders_billing.PNG" width='75%' />
RED: Bounced session<br>
Green: Non-bounced session<br>

Finally, I can wrap this in a subquery and aggregate the results to calculate the conversion rate for both pages.
<img src="Images/12C-billing_conv_rt.PNG" width='75%' />
# Conclusion:
As I suspected, the addition of the '/billing-2' page has overwhelming proved to be a better conversion factor for the website.

# REVENUE GROWTH PER BILLING PAGE SESSION
# Key question 13:

What’s the revenue per billing page session for the new billing page versus the old billing page?

Since the sites new billing page has improved conversion rates by 17%, I want to know how much it contributes to the company's top-line growth. 
My analysis will start from the launch date of '/billing-2' to the end of the year.

First, I begin with a left join on the ‘website-pageviews’ table with the ‘orders’ table. The 'orders' table contains revenue generated for each order so I'll bring that over as well to calculate the revenue-per-billing-page.
<img src="Images/14A_revenue_billing_page.PNG" width='70%' />
This is a session level table with the ability to segment based on the billing version seen and I can pull the success metrics based on wether or not the order id populated.
 
Making this table a subquery, I will pull the 'billing_page_version_seen' which will be used in the 'group by' clause. The metrics used alongside the 'group by' will be a 'count' 'distinct' call on the 'website_session_id' as well as a 'sum' function on the 'price_usd' divided by a 'count' 'distinct' call on the 'website_session_id' column to create the 'revenue-per-billing-page-seen' metric.
<img src="Images/14B.PNG" width='70%' /><br>

<br>--$22.40 revenue per billing page seen for the old version
--$31.02 for the new version<br>
<br>--Lift: $8.62 per billing page view

# Conclusion

Every time a customer views the billing page, the company is making $8.62 more than previously.

Just looking at last month's total 'billing' sessions of '919', I can approximate the billing page test brought nearly $8,000 more in revenue for the company's top-line growth.